# BEFORE YOU START: READ THIS

1. install required packages
2. get your API key from alpha vantage
3. run each cell through CHAT GPT or search in Google to see what eat term means
4. right now, all i have done is download APPLE data and do feature engineering on it.

# NEXT STEPS:
5. Need to create functions to do the feature engineering on 100 stocks for example
6. Need to create the model to identify 10 stocks to hold and identify buy/sell triggers
7. Need to find NEWS API to include in the model.

In [42]:
# Install required packages in the folllowing few cells

In [43]:
%%capture
 # capture hides output
!pip install yfinance pandas numpy scikit-learn ta pandas_ta xgboost

In [44]:
%%capture
# hidden output
!pip install alpha_vantage

In [45]:
%%capture
# hidden output, !pip install is required to run just on a machine
!pip install requests pandas_datareader


#### IMPORTS

In [46]:
import numpy as np
import pandas as pd
from alpha_vantage.timeseries import TimeSeries
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime
from datetime import time
from datetime import timedelta
from pandas.tseries.offsets import BDay
import matplotlib.lines as mlines
from dotenv import load_dotenv
import os
import matplotlib.patches as patches
import seaborn as sns
import warnings
import requests
import json
import pandas_ta as ta
from ta import trend, volatility, momentum, volume


from numpy import array
from importlib import reload # to reload modules if we made changes to them without restarting kernel


### ALPHA VANTAGE DATA

#### AAPL TICKER


In [47]:
# get your API key from the .env file
url = 'https://www.alphavantage.co/query?function=CASH_FLOW&symbol=AAPL&apikey='+os.getenv('AV_API_KEY')
r = requests.get(url)
data = r.json()

print(json.dumps(data,indent=4))

{
    "symbol": "AAPL",
    "annualReports": [
        {
            "fiscalDateEnding": "2023-09-30",
            "reportedCurrency": "USD",
            "operatingCashflow": "110543000000",
            "paymentsForOperatingActivities": "5703000000",
            "proceedsFromOperatingActivities": "None",
            "changeInOperatingLiabilities": "1142000000",
            "changeInOperatingAssets": "7719000000",
            "depreciationDepletionAndAmortization": "11519000000",
            "capitalExpenditures": "10959000000",
            "changeInReceivables": "417000000",
            "changeInInventory": "1618000000",
            "profitLoss": "96995000000",
            "cashflowFromInvestment": "3705000000",
            "cashflowFromFinancing": "-108488000000",
            "proceedsFromRepaymentsOfShortTermDebt": "-7956000000",
            "paymentsForRepurchaseOfCommonStock": "77550000000",
            "paymentsForRepurchaseOfEquity": "77550000000",
            "paymentsForRepurch

In [48]:
#json file, we can extract detailed data we need for a company
apple_annual_reports_json = data["annualReports"]
apple_quarterly_reports_json = data["quarterlyReports"]

apple_annual_reports_json

[{'fiscalDateEnding': '2023-09-30',
  'reportedCurrency': 'USD',
  'operatingCashflow': '110543000000',
  'paymentsForOperatingActivities': '5703000000',
  'proceedsFromOperatingActivities': 'None',
  'changeInOperatingLiabilities': '1142000000',
  'changeInOperatingAssets': '7719000000',
  'depreciationDepletionAndAmortization': '11519000000',
  'capitalExpenditures': '10959000000',
  'changeInReceivables': '417000000',
  'changeInInventory': '1618000000',
  'profitLoss': '96995000000',
  'cashflowFromInvestment': '3705000000',
  'cashflowFromFinancing': '-108488000000',
  'proceedsFromRepaymentsOfShortTermDebt': '-7956000000',
  'paymentsForRepurchaseOfCommonStock': '77550000000',
  'paymentsForRepurchaseOfEquity': '77550000000',
  'paymentsForRepurchaseOfPreferredStock': 'None',
  'dividendPayout': '15025000000',
  'dividendPayoutCommonStock': '15025000000',
  'dividendPayoutPreferredStock': 'None',
  'proceedsFromIssuanceOfCommonStock': 'None',
  'proceedsFromIssuanceOfLongTermDebt

In [49]:
#convert json to dataframe
apple_annual_reports= pd.DataFrame(apple_annual_reports_json)
apple_quarterly_reports = pd.DataFrame(apple_quarterly_reports_json)

#APPL quarterly
apple_quarterly_reports

,fiscalDateEnding,reportedCurrency,operatingCashflow,paymentsForOperatingActivities,proceedsFromOperatingActivities,changeInOperatingLiabilities,changeInOperatingAssets,depreciationDepletionAndAmortization,capitalExpenditures,changeInReceivables,...,dividendPayoutCommonStock,dividendPayoutPreferredStock,proceedsFromIssuanceOfCommonStock,proceedsFromIssuanceOfLongTermDebtAndCapitalSecuritiesNet,proceedsFromIssuanceOfPreferredStock,proceedsFromRepurchaseOfEquity,proceedsFromSaleOfTreasuryStock,changeInCashAndCashEquivalents,changeInExchangeRate,netIncome
0,2023-12-31,USD,39895000000,None,None,-8407000000,-9530000000,2848000000,2392000000,-11124000000,...,3825000000,None,None,0,None,-20139000000,None,None,None,33916000000
1,2023-09-30,USD,21598000000,3113000000,None,14946000000,21006000000,2653000000,2163000000,21137000000,...,3758000000,None,0,0,None,-21003000000,None,839000000,None,22956000000
2,2023-06-30,USD,26380000000,None,None,None,None,3052000000,2093000000,-3661000000,...,3849000000,None,0,0,None,0,None,2769000000,None,19881000000
3,2023-03-31,USD,28560000000,None,None,None,None,2898000000,2916000000,17786000000,...,3650000000,None,0,0,None,0,None,5155000000,None,24160000000
4,2022-12-31,USD,34005000000,703000000,None,-2186000000,-689000000,2916000000,3787000000,-6595000000,...,3768000000,None,0,0,None,-19475000000,None,-3003000000,None,29998000000
5,2022-09-30,USD,24127000000,2755000000,None,21420000000,21468000000,2865000000,3289000000,18693000000,...,3703000000,None,None,5465000000,None,-24428000000,None,-3884000000,None,20721000000
6,2022-06-30,USD,22892000000,504000000,None,-6627000000,-3402000000,2805000000,2102000000,-3165000000,...,3811000000,None,None,0,None,-21865000000,None,19442000000,None,19442000000
7,2022-03-31,USD,28166000000,875000000,None,-23746000000,-21694000000,2737000000,2514000000,-19931000000,...,3595000000,None,None,0,None,-22631000000,None,25010000000,None,25010000000
8,2021-12-31,USD,46966000000,531000000,None,24511000000,17986000000,2697000000,2803000000,13746000000,...,3732000000,None,None,None,None,-20478000000,None,2701000000,None,34630000000
9,2021-09-30,USD,20200000000,2217000000,None,19386000000,21176000000,2989000000,3223000000,17604000000,...,3640000000,None,544000000,6470000000,None,-19204000000,None,653000000,None,20551000000


In [50]:
# Function to get stock data from Alpha Vantage

def getalphastock(api_key, stockname='msft',timeint='1min'):

    ts = TimeSeries(key=api_key, output_format='pandas')
    data, meta_data = ts.get_intraday(symbol=stockname,interval=timeint,outputsize='full')
    return data

In [51]:
import yfinance as yf
from pandas_datareader import data as pdr
# Function to get stock data from Yahoo Finance

def getyahooticks(tickersin="AAPL MSFT", periodin = "1d", intervalin="5m"):
    yf.pdr_override()

    data = pdr.get_data_yahoo(
            # tickers list or string as well
            tickers = tickersin,

            # use "period" instead of start/end
            # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
            # (default is '1mo')
            period = periodin,

            # fetch data by interval (including intraday if period < 60 days)
            # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
            # (optional, default is '1d')
            interval = intervalin,

            # group by ticker (to access via data['SPY'])
            # (optional, default is 'column')
            group_by = 'ticker',

            # adjust all OHLC automatically
            # (optional, default is False)
            auto_adjust = True,

            # download pre/post regular market hours data
            # (optional, default is False)
            prepost = True,

            # use threads for mass downloading? (True/False/Integer)
            # (optional, default is True)
            threads = True,

            # proxy URL scheme use use when downloading?
            # (optional, default is None)
            proxy = None
        )
    return data

In [62]:

# Access your API key from the environment variables
api_key = os.getenv('AV_API_KEY')

#Get Data
ts = TimeSeries(key=api_key, output_format='pandas')

stockname = 'AAPL'
period = 5
freqfix = '5min'
datainter = '5m'

#pick data source
fromdata='alpha'

if fromdata == 'alpha':
    data = getalphastock(api_key,stockname,'5min')
    stockpricefull = getalphastock(api_key,stockname)
    stockpricefull.columns = ['Open','High','Low','Close','Volume']
    stock = data.copy()
    stock.columns = ['Open','High','Low','Close','Volume']
    stockpricefull.head()

elif fromdata=='yahoo':
    stock = getyahooticks(tickersin=stockname, periodin = "5d", intervalin=datainter)
    #stockpricefull = getyahooticks(tickersin=stockname, periodin = "30d", intervalin="5m")
    stockpricefull= stock.copy()
    stock.index.name='date'
    stockpricefull.index.name='date'
    stock.index = stock.index.tz_localize(None)#.tz_convert(tz = None)
    stockpricefull.index = stockpricefull.index.tz_localize(None)
    #stockpricefull.index.to_datetime

print(stockpricefull)

                       Open    High     Low   Close  Volume
date                                                       
2024-04-26 19:59:00  169.63  169.70  169.61  169.66   261.0
2024-04-26 19:58:00  169.61  169.64  169.60  169.63  1161.0
2024-04-26 19:57:00  169.64  169.64  169.60  169.62   759.0
2024-04-26 19:56:00  169.65  169.66  169.60  169.62   688.0
2024-04-26 19:55:00  169.69  169.70  169.60  169.63   206.0
...                     ...     ...     ...     ...     ...
2024-04-01 04:04:00  171.86  171.86  171.79  171.83   580.0
2024-04-01 04:03:00  171.79  171.86  171.74  171.83   354.0
2024-04-01 04:02:00  171.70  171.79  171.56  171.79  1823.0
2024-04-01 04:01:00  171.65  171.76  171.49  171.70  1159.0
2024-04-01 04:00:00  172.02  172.02  171.40  171.52  3307.0

[19139 rows x 5 columns]


# Source Data

In [81]:
import yfinance as yf
import pandas as pd
import time

# Define the function to get the sector for each stock
def get_sector(ticker):
    try:
        stock_info = yf.Ticker(ticker).info
        return stock_info.get('sector', 'N/A')  # Returns 'N/A' if sector information is not available
    except Exception as e:
        print(f"Could not fetch sector for {ticker}: {e}")
        return 'N/A'
    

# get ticker symbols
tickers = pd.read_csv('data/tickers.csv')
tickers = tickers['Symbol'].tolist()

sectors = []

# Loop through each ticker and fetch the sector
for ticker in tickers:
    print(f"Fetching sector for {ticker}")
    sector = get_sector(ticker)
    sectors.append(sector)
   # time.sleep(0.5)  # Delay to avoid hitting API rate limits


Fetching sector for MSFT
Fetching sector for AAPL
Fetching sector for NVDA
Fetching sector for AMZN
Fetching sector for GOOGL
Fetching sector for META
Fetching sector for GOOG
Fetching sector for BRK.B
Fetching sector for AVGO
Fetching sector for LLY
Fetching sector for JPM
Fetching sector for XOM
Fetching sector for TSLA
Fetching sector for UNH
Fetching sector for V
Fetching sector for MA
Fetching sector for PG
Fetching sector for JNJ
Fetching sector for HD
Fetching sector for MRK
Fetching sector for COST
Fetching sector for CVX
Fetching sector for ABBV
Fetching sector for CRM
Fetching sector for BAC
Fetching sector for WMT
Fetching sector for AMD
Fetching sector for NFLX
Fetching sector for PEP
Fetching sector for KO
Fetching sector for TMO
Fetching sector for ADBE
Fetching sector for WFC
Fetching sector for LIN
Fetching sector for DIS
Fetching sector for MCD
Fetching sector for CSCO
Fetching sector for ACN
Fetching sector for ORCL
Fetching sector for ABT
Fetching sector for QCOM
Fet

In [85]:
set(sectors)


{'Basic Materials',
 'Communication Services',
 'Consumer Cyclical',
 'Consumer Defensive',
 'Energy',
 'Financial Services',
 'Healthcare',
 'Industrials',
 'N/A',
 'Real Estate',
 'Technology',
 'Utilities'}

In [87]:
# get tickers
tickers = pd.read_csv('data/tickers.csv')

tickers['Sector'] = sectors

# Check the result
print(tickers.head())

# Save the result to a CSV file
tickers.to_csv('data/tickers.csv', index=False)

   #                Company Symbol Portfolio%                  Sector
0  1         Microsoft Corp   MSFT      7.05%              Technology
1  2             Apple Inc.   AAPL      5.74%              Technology
2  3            Nvidia Corp   NVDA      5.06%              Technology
3  4         Amazon.com Inc   AMZN      3.83%       Consumer Cyclical
4  5  Alphabet Inc. Class A  GOOGL      2.37%  Communication Services


In [89]:
#get ticker symbols
ticker_symbols = tickers['Symbol'].tolist()

In [90]:
# Split tickers into batches of 100 for we don't hit the API rate limit
ticker_batches = [ticker_symbols[i:i + 100] for i in range(0, len(ticker_symbols), 100)]

# Start and end dates
start_date = '2000-01-01'
end_date = '2024-04-01'

# Create an empty DataFrame to store all data
all_data = pd.DataFrame()

# Loop through each batch of tickers and fetch the historical data
for batch in ticker_batches:
    for ticker in batch:
        print(f"Downloading data for {ticker}")
        data = yf.download(ticker, start=start_date, end=end_date)
        data.dropna(inplace=True)  # Clean data by dropping NAs
        data['Ticker'] = ticker  # Add a column to identify the stock
        all_data = pd.concat([all_data, data], axis=0)
    
    # Save to CSV after each batch is processed
    all_data.to_csv('data/historical_stock_data_partial.csv')

    time.sleep(10)  # sleep tim to avoid hitting API rate limits

# Save the final result to a CSV file
all_data.to_csv('data/historical_stock_data_final.csv')

print("All data downloaded and saved.")


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


...

[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[*********************100%%**********************]  1 of 1 completed

All data downloaded and saved.


# FEATURE ENGINEERING


### TECHNICAL INDICATORS:

* MACD (Moving Average Convergence Divergence)
* RSI (Relative Strength Index)
* Bollinger Bands High (the low is commented out)
* ADX (Average Directional Index)
* ATR (Average True Range)
* CCI (Commodity Channel Index)
* MFI (Money Flow Index)
* Stochastic Oscillator
* Keltner Channel
* VWAP (Volume Weighted Average Price)
* EOM (Ease Of Movement)
* VPT (Volume Price Trend)

In [57]:
# 1. Download daily OHLCV for APPLE stock
data_yf = yf.download('AAPL', start='2008-01-01', end='2024-04-25')

# Preserve the original OHLCV data_yf
ohlcv_data_yf = data_yf.copy()
#(Open/High/Low/Close/Volume) data 

# 2. Clean data_yf: DROP NAs
data_yf.dropna(inplace=True)

# 3. Add technical indicators

data_yf['macd'] = trend.MACD(data_yf['Close']).macd()
data_yf['macd_signal'] = trend.MACD(data_yf['Close']).macd_signal()
data_yf['rsi'] = momentum.RSIIndicator(data_yf['Close']).rsi()
data_yf['bb_high'] = volatility.BollingerBands(data_yf['Close']).bollinger_hband()
#data_yf['bb_low'] = volatility.BollingerBands(data_yf['Close']).bollinger_lband()
data_yf['adx'] = trend.ADXIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).adx()
data_yf['atr'] = volatility.AverageTrueRange(data_yf['High'], data_yf['Low'], data_yf['Close']).average_true_range()
data_yf['cci'] = trend.CCIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close']).cci()
data_yf['mfi'] = volume.MFIIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).money_flow_index()
data_yf['so'] = momentum.StochasticOscillator(data_yf['High'], data_yf['Low'], data_yf['Close']).stoch()
# Add additional indicators as needed...
# Calculate additional indicators
data_yf['kc'] = volatility.KeltnerChannel(data_yf['High'], data_yf['Low'], data_yf['Close']).keltner_channel_mband()
data_yf['vwap'] = volume.VolumeWeightedAveragePrice(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).volume_weighted_average_price()
data_yf['eom'] = volume.EaseOfMovementIndicator(data_yf['High'], data_yf['Low'], data_yf['Close'], data_yf['Volume']).ease_of_movement()
data_yf['vpt'] = volume.VolumePriceTrendIndicator(data_yf['Close'], data_yf['Volume']).volume_price_trend()



# Momentum Indicators
data_yf.ta.rsi(append=True)
data_yf.ta.mfi(append=True)
data_yf.ta.roc(append=True)
data_yf.ta.willr(append=True)

# Volume Indicators
data_yf.ta.obv(append=True)



# Volatility Indicators
# pandas_ta doesn't seem to have NATR equivalent, but ATR is available

# Average True Range (ATR)
# ATR is a technical analysis indicator that measures market volatility by decomposing the entire range of an asset price for that period.
# It is commonly used to understand the volatility of a stock, commodity, forex, or crypto over a given period.
# The ATR is particularly useful in systems that need to place actual numerical values to the volatility or potential price change of an asset, often used to adjust trading systems and stop-loss orders.

# Calculate ATR and add it to the DataFrame
data_yf['atr'] = data_yf.ta.atr()

# 4. Create Buy/Sell signals based on the technical indicators

# Either use the indicator values directly or create signals based on the indicator values
# depending on the strategy, we can create signals for each indicator!!
# these signals are used to create a trading strategy and are very subjective; they might not even be useful
# we can get rid of them.

# SIGNALS for each indicator
# Calculate the Force Index (FI)
data_yf['fi'] = (data_yf['Close'].diff()) * data_yf['Volume']

# Create a Buy/Sell/Hold Signal for FI
data_yf['fi_buy_signal'] = np.where(data_yf['fi'] > 0, 1, 0)  # Buy if FI is positive
data_yf['fi_sell_signal'] = np.where(data_yf['fi'] < 0, 1, 0)  # Sell if FI is negative

# MACD signal
data_yf['macd_buy_signal'] = np.where(data_yf['macd'] > 0, 1, 0)
data_yf['macd_sell_signal'] = np.where(data_yf['macd'] < 0, 1, 0)

# RSI signal
data_yf['rsi_buy_signal'] = np.where(data_yf['rsi'] < 30, 1, 0)
data_yf['rsi_sell_signal'] = np.where(data_yf['rsi'] > 70, 1, 0)

# Stochastic Oscillator signal
data_yf['so_buy_signal'] = np.where(data_yf['so'] < 20, 1, 0)
data_yf['so_sell_signal'] = np.where(data_yf['so'] > 80, 1, 0)

# CCI signal
data_yf['cci_buy_signal'] = np.where(data_yf['cci'] < -100, 1, 0)
data_yf['cci_sell_signal'] = np.where(data_yf['cci'] > 100, 1, 0)

# MFI signal
data_yf['mfi_buy_signal'] = np.where(data_yf['mfi'] < 20, 1, 0)
data_yf['mfi_sell_signal'] = np.where(data_yf['mfi'] > 80, 1, 0)

# WILLR signal
data_yf['WILLR_buy_signal'] = np.where(data_yf['WILLR_14'] < -80, 1, 0)
data_yf['WILLR_sell_signal'] = np.where(data_yf['WILLR_14'] > -20, 1, 0)


# Other indicators such as ATR, EOM, VWAP, etc., may not have clear buy/sell interpretations.
# Depending on strategy: might want to create signals for these as well.

# If suing signals, drop the original indicator columns
# if not using signals, keep the original indicator columns

# Drop the original technical indicator columns, keeping only the signal columns
indicator_cols_to_drop = [
    'macd', 'macd_signal', 'rsi', 'bb_high', 'adx', 'atr', 'cci', 'mfi', 'so', 'kc', 'fi', 'vwap', 'eom', 'vpt',
    'WILLR_14', 'OBV' # pandas_ta indicators
]
data_yf.drop(columns=indicator_cols_to_drop, inplace=True)

[*********************100%%**********************]  1 of 1 completed


In [58]:
data_yf.columns


Index(['Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'RSI_14',
       'MFI_14', 'ROC_10', 'fi_buy_signal', 'fi_sell_signal',
       'macd_buy_signal', 'macd_sell_signal', 'rsi_buy_signal',
       'rsi_sell_signal', 'so_buy_signal', 'so_sell_signal', 'cci_buy_signal',
       'cci_sell_signal', 'mfi_buy_signal', 'mfi_sell_signal',
       'WILLR_buy_signal', 'WILLR_sell_signal'],
      dtype='object')

#### MOVING AVERAGES

In [59]:
# Create lagged features for each indicator for the previous 5 days
indicator_cols = [col for col in data_yf.columns if col not in ['win', 'gain_loss_pct', 'Open', 'High', 'Low', 'Close', 'Volume','Adj Close']]

for col in indicator_cols:
    for i in range(1, 6):
        data_yf[f'{col}_lag_{i}'] = data_yf[col].shift(i)

# Remove rows with NaN values after shifting
data_yf.dropna(inplace=True)


In [60]:
# 4. Calculate gain/loss percentage and assign -1, 0, 1
data_yf['gain_loss_pct'] = (data_yf['Close'] - data_yf['Open']) / data_yf['Open']
data_yf['win'] = np.where(data_yf['gain_loss_pct'] > 0.005, 1, np.where(data_yf['gain_loss_pct'] < -0.0025, 0, 0))

# Drop OHLCV columns
data_yf.drop(columns=['Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close'], inplace=True)


In [61]:
data_yf.shape, data_yf.columns, data_yf.describe()

((4087, 104),
 Index(['RSI_14', 'MFI_14', 'ROC_10', 'fi_buy_signal', 'fi_sell_signal',
        'macd_buy_signal', 'macd_sell_signal', 'rsi_buy_signal',
        'rsi_sell_signal', 'so_buy_signal',
        ...
        'WILLR_buy_signal_lag_3', 'WILLR_buy_signal_lag_4',
        'WILLR_buy_signal_lag_5', 'WILLR_sell_signal_lag_1',
        'WILLR_sell_signal_lag_2', 'WILLR_sell_signal_lag_3',
        'WILLR_sell_signal_lag_4', 'WILLR_sell_signal_lag_5', 'gain_loss_pct',
        'win'],
       dtype='object', length=104),
             RSI_14       MFI_14       ROC_10  fi_buy_signal  fi_sell_signal  \
 count  4087.000000  4087.000000  4087.000000    4087.000000     4087.000000   
 mean     54.996555    53.539006     1.030595       0.525569        0.472718   
 std      13.171973    16.657557     5.901597       0.499407        0.499316   
 min      16.550367     0.000000   -32.737052       0.000000        0.000000   
 25%      45.075708    41.291604    -2.478150       0.000000        0.000000  

Next steps:

1. create functions to request a list of 100 stocks and get comprehensive data from alpha vantage
2. compare the data sources between AV and YF and decide what info to merge
3. decide how to incorporate the financials from each stock (meaning we get quarterly data for example for APPLE in the dataframe: `apple_quarterly_reports` ) this data has dimensions: 58x29 , therefore not sure how to incorporate it whith dataframe 15306 x 5


In [ ]:
#Need a train loader and a test loader
#Need to convert to optDataset, then wrap with DataLoader

# COMBINING DATA 

In [65]:
apple_data = pd.merge_asof(stockpricefull.sort_index(), data_yf.sort_index(), left_index=True, right_index=True, direction='nearest')

In [69]:
apple_data.head()

,Open,High,Low,Close,Volume,RSI_14,MFI_14,ROC_10,fi_buy_signal,fi_sell_signal,...,WILLR_buy_signal_lag_3,WILLR_buy_signal_lag_4,WILLR_buy_signal_lag_5,WILLR_sell_signal_lag_1,WILLR_sell_signal_lag_2,WILLR_sell_signal_lag_3,WILLR_sell_signal_lag_4,WILLR_sell_signal_lag_5,gain_loss_pct,win
date,,,,,,,,,,,,,,,,,,,,,
2024-04-01 04:00:00,172.02,172.02,171.40,171.52,3307.0,40.168189,33.69034,-1.500403,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006776,0
2024-04-01 04:01:00,171.65,171.76,171.49,171.70,1159.0,40.168189,33.69034,-1.500403,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006776,0
2024-04-01 04:02:00,171.70,171.79,171.56,171.79,1823.0,40.168189,33.69034,-1.500403,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006776,0
2024-04-01 04:03:00,171.79,171.86,171.74,171.83,354.0,40.168189,33.69034,-1.500403,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006776,0
2024-04-01 04:04:00,171.86,171.86,171.79,171.83,580.0,40.168189,33.69034,-1.500403,0,1,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.006776,0


In [ ]:
!pip install PyPl gurobipy pyepo

In [ ]:
import torch.nn.functional as F
from torch import nn
import torch 
class StockLSTM(nn.Module):

    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers=1):
        super(StockLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,num_layers=num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, x):
        x = self.embedding(x)
        all_h, (h, c) = self.lstm(x)
        out = self.fc(all_h) # Apply Linear layer to outputs from all the hidden state.
        return out

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pyepo
from pyepo.model.grb import optGrbModel
import torch
from torch import nn
from torch.utils.data import DataLoader

m = 50 # change based on number of assets
cov = np.cov(np.random.randn(10, m), rowvar=False) # covariance matrix
optmodel = pyepo.model.grb.portfolioModel(m, cov) # build model

Auto-Sklearn cannot be imported.
Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
import time

# train model
def trainModel(reg, loss_func, method_name, num_epochs=20, lr=1e-2):
    # set adam optimizer
    optimizer = torch.optim.Adam(reg.parameters(), lr=lr)
    # train mode
    reg.train()
    # init log
    train_loss_log = []
    loss_log_regret = [pyepo.metric.regret(reg, optmodel, loader_test)]
    # init elpased time
    elapsed = 0
    for epoch in range(num_epochs):
        # start timing
        tick = time.time()
        # load data
        train_loss = 0
        for i, data in enumerate(loader_train):
            x, c, w, z = data
            # cuda
            if torch.cuda.is_available():
                x, c, w, z = x.cuda(), c.cuda(), w.cuda(), z.cuda()
            # forward pass
            cp = reg(x)
            if method_name == "spo+":
                loss = loss_func(cp, c, w, z)
            elif method_name == "mse":
                loss = loss_func(cp, c)
            else:
                raise ValueError("Method name {} not supported".format(method_name))
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            # record time
            tock = time.time()
            elapsed += tock - tick
            train_loss += loss.item()
        train_loss /= len(loader_train)
        train_loss_log.append(train_loss)
        regret = pyepo.metric.regret(reg, optmodel, loader_test)
        loss_log_regret.append(regret)
        print("Epoch {:2},  Loss: {:9.4f},  Regret: {:7.4f}%".format(epoch+1, train_loss, regret*100))
    print("Total Elapsed Time: {:.2f} Sec.".format(elapsed))
    return train_loss_log, loss_log_regret

In [ ]:

spop = pyepo.func.SPOPlus(optmodel, processes=1)

In [ ]:
# Hyperparameters
VOCAB_SIZE = #fill in once data is imported
EMBEDDING_DIM = #param we can optimize
HIDDEN_DIM = #also a param to optimize
learning_rate = #another optimizable param
epoch = #optimize
num_layers = #optimize
epochs = 20
learning_rate = 2e-3
method_name = "spo+"

# Instantiate the model
lstm = StockLSTM(VOCAB_SIZE, EMBEDDING_DIM, HIDDEN_DIM,num_layers=num_layers)
loss_log_lstm_spo, loss_log_regret_lstm_spo = trainModel(lstm, loss_func=spop, method_name=method_name)
